# Introductory Machine Learning: Assignment 5

**Deadline:**

Assignment 5 is due Thursday, November 30 at 11:59pm. Late work will not be accepted as per the course policies (see the Syllabus and Course policies on [Canvas](https://canvas.yale.edu).

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged.

You should start early so that you have time to get help if you're stuck. The drop-in office hours schedule can be found on [Canvas](https://canvas.yale.edu).  You can also post questions or start discussions on [Ed Discussion](https://edstem.org/us/courses/44592/discussion). The problems are broken up into steps that should help you to make steady progress.

**Submission:**

Submit your assignment as a .pdf on Gradescope, and as a .ipynb on Canvas. You can access Gradescope through Canvas on the left-side of the class home page. The problems in each homework assignment are numbered. Note: When submitting on Gradescope, please select the correct pages of your pdf that correspond to each problem. This will allow graders to find your complete solution to each problem.

To produce the .pdf, please do the following in order to preserve the cell structure of the notebook:  
1.  Go to "File" at the top-left of your Jupyter Notebook
2.  Under "Download as", select "HTML (.html)"
3.  After the .html has downloaded, open it and then select "File" and "Print" (note you will not actually be printing)
4.  From the print window, select the option to save as a .pdf

**Topics**
1. Bayesian inference
2. Topic models
3. Neural networks
4. Reinforcement learning

The first problem tests some of the basics of Bayesian inference and topic models. The second problem has you building topic models to improve pricing of houses on Zillow. The third problem gives you experience with neural networks in tensorflow. The fourth problem is a reinforcement learning task similar to the Taxi problem, but with a random environment. 

Note: The assignment looks longer than it really is. We step you through most of the code that you need. But it's still on the long side. Although the assignment is due in three weeks, we encourage you to start early!

## Problem 1. Toy Story: Bayesian calculations (12 points)

Gibbs sampling is one of the commonly used approach to approximate the inference for Latent Dirichlet Allocation model. In this problem, we will use the toy example from class.

<img src="https://raw.githubusercontent.com/YData123/sds265-fa23/main/assignments/assn5/diagram.png" width="500" align="center">

Assume that there are 3 documents and 15 words in the corpus. We would like to build a topic model with 3 topics. The proportions parameter is $\alpha$ and the topic parameter is $\eta$. The table below shows an assignment of topics to words in the toy corpus at one stage of the Gibbs sampling algorithm. 

<img src="https://raw.githubusercontent.com/YData123/sds265-fa23/main/assignments/assn5/words.png" width="500" align="center">

Using only these assignment id $Z$ for each word, the following problems ask you 
to calculate the posterior topic proportions for each document, and word probabilities 
for one word in each of the three topics. To answer these questions you only need
to use the basic properties of the Dirichlet distribution as a prior for 
a multinomial, as presented in class (and in the notes on Bayesian inference).


### Problem 1.1: Per-document topic proportions

Given the $Z$ values in the table, what are the posterior distributions of $\theta_{d}$ for documents $D_{1}$, $D_{2}$ and $D_{3}$ from left to right. Assume the prior over $\theta$ is 
$\mbox{Dirichlet}(\alpha, \alpha, \alpha)$.

[your markdown here]

### Problem 1.2: Topics

Here are the 15 words in our corpus:

addiction, brother, baseball, catcher, daughter, divorce, drug, hit, inning, illegal, meth, mother, father, son, steroids

What is the posterior mean for the probability $p(\mbox{addiction} | \mbox{topic 1})$? 
Assume that the prior distribution over the topics is $\mbox{Dirichlet}(\eta,...\eta)$.

[your markdown here]

What is the posterior mean of the probability $p(\mbox{baseball}| \mbox{topic 2})$?

[your markdown here]

What is the posterior mean of the probability $p(\mbox{divorce} | \mbox{topic 3})$?

[your markdown here]

## Problem 2. Up: Rising house prices  (30 points)

![zillow](https://raw.githubusercontent.com/YData123/sds265-fa23/main/assignments/assn5/zillow.png)

### Overview of the problem

Here we have a dataset of single family houses sold in Connecticut near the beginning of 2021, collected from [Zillow](https://www.zillow.com/homes/connecticut_rb/). You will build linear models of the price for which each house sold, based on its characteristics given in the real estate listing. Such characteristics include internal square footage, the year it was built, the bedroom count, the bathroom count, and the area of the lot. 

But there is also usually a lengthy description written by the real estate agent. Is there any additional information hidden in this description that would help improve the model of the price? This is the question we focus on in this problem.

Answering such a question is difficult because the description is written in natural language with thousands of different words. Here we use topic models as a dimension reduction technique. Specifically, instead of using thousands of possible words, and how many times they show up in each house description, we reduce the words to the topic proportions $\theta_d$ for each document, obtained by posterior inference. These proportions are combined with the other quantitative variables in a linear model with the logarithm of the house price as the response variable. 

*Important note:* At first glance, this problem looks really long. But this is deceiving. 
After reading in the data, we have you make some plots of the log-transformed variables. 
After that, you just need to run the code that leads up to training a 10-topic topic model, 
and fitting a linear model using the resulting topic proportions. After this, you are asked to compare the results to those obtained with a 3-topic model. To do this, you can simply copy the code used for the 10-topic model. After that, the crux of the problem is to analyze, understand, and describe the results.

Acknowledgment: The data were scraped and the analysis was done by [Parker Holzer](https://parkerholzer.github.io/), as he began his search for a new house for his family after beginning a job as a data scientist. Thanks Parker!


In [ ]:
import numpy as np
import pandas as pd
import re
import gensim
from collections import Counter
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

### Read in and clean up the data

In [ ]:
import pandas as pd
ct_homes = pd.read_csv('https://raw.githubusercontent.com/YData123/sds265-fa23/main/assignments/assn5/ct_zillow.csv')
ct_homes

#### Transform the data

We add columns to `ct_homes` called `logAREA`, `logLOTSIZE`, and `logPRICE` that take the logarithms of the corresponding columns in the original data. 


In [ ]:
ct_homes['logAREA'] = np.log(ct_homes['AREA'])
ct_homes['logLOTSIZE'] = np.log(ct_homes['LOTSIZE'])
ct_homes['logPRICE'] = np.log(ct_homes['PRICE'])
ct_homes

#### 2.1 Plot the data 

1. Show histograms of each of the log-transformed columns.

1. Our regression models will use these transformed values. Why might it be preferable to use the logarithms rather than the original data? Explain.


[Your code and markdown here]

Let's look at one of the descriptions as an example.

In [ ]:
example = 9
ct_homes["DESCRIPTION"][example]

#### Helper functions

The following two functions will be used to clean up the text a bit and separate into tokens

In [ ]:
def cleanup_description(desc):
    if type(desc) == float:
        desc = ""
    words = [re.sub(r'[^a-z]', '', w) for w in desc.lower().split(' ')]
    return ' '.join(words)

def reduce_to_vocabulary(desc, vocab):
    return ' '.join([w for w in cleanup_description(desc).split(' ') if w in vocab])


In [ ]:
cleanup_description(ct_homes['DESCRIPTION'][example])


#### Next we build a vocabulary of words

In [ ]:
vocab = Counter()
for dsc in ct_homes['DESCRIPTION']:
    vocab.update(cleanup_description(dsc).split(' '))


In [ ]:
print("Number of unique tokens: %d" % len(vocab))

#### Remove words that are either too common or too rare

In [ ]:
vocab = Counter(token for token in vocab.elements() if vocab[token] > 5)
stop_words = [item[0] for item in vocab.most_common(50)]
vocab = Counter(token for token in vocab.elements() if token not in stop_words)
print("Number of unique tokens: %d" % len(vocab))

#### Build a mapping between unique words and integers

In [ ]:
desc = ct_homes['DESCRIPTION'][example]
print('Original description:\n---------------------')
print(desc)

print('\nCleaned up text:\n----------------')
print(cleanup_description(desc))

print('\nReduced to vocabulary:\n----------------------')
print(reduce_to_vocabulary(desc, vocab))

#### Build a mapping between unique words and integers

In [ ]:
id2word = {idx: pair[0] for idx, pair in enumerate(vocab.items())}
word2id = {pair[0]: idx for idx, pair in enumerate(vocab.items())}

s = 'nyc'
print("Number of tokens mapped: %d" % len(id2word))
print("Identifier for '%s': %d" % (s,word2id[s]))
print("Word for identifier %d: %s" % (word2id[s], id2word[word2id[s]]))

#### Map to word id format

Now, use the format required to build a language model, mapping each word to its id, 

In [ ]:
tokens = []
for dsc in ct_homes['DESCRIPTION']:
    clean = reduce_to_vocabulary(cleanup_description(dsc), vocab)
    toks = clean.split(' ')
    tokens.append(toks)

In [ ]:
corpus = []
for toks in tokens:
    tkn_count = Counter(toks)
    corpus.append([(word2id[item[0]], item[1]) for item in tkn_count.items()])
    
dsc = ct_homes['DESCRIPTION'][example]
clean = reduce_to_vocabulary(cleanup_description(dsc), vocab)
toks = clean.split(' ')
print("Abstract, tokenized:\n", toks, "\n")
print("Abstract, in corpus format:\n", corpus[10])

#### Build a Topic Model with 10 topics

Note: Don't worry about the various settings used in the call to `LdaModel`. If you want to read up on these, just check out the documentation. 


In [ ]:
%%time
tm = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                     id2word=id2word,
                                     num_topics=10, 
                                     random_state=100,
                                     chunksize=100,
                                     passes=10,
                                     alpha='auto',
                                     per_word_topics=True)

In [ ]:
num_topics = 10
num_words = 15
top_words = pd.DataFrame({'word rank': np.arange(1,num_words+1)})
for k in np.arange(num_topics): 
    topic = tm.get_topic_terms(k, num_words)
    words = [id2word[topic[i][0]] for i in np.arange(num_words)]
    probs = [topic[i][1] for i in np.arange(num_words)]
    top_words['topic %d' % k] = words

top_words

In [ ]:
topic_dist = tm.get_document_topics(corpus[example])
topics = [pair[0] for pair in topic_dist]
probabilities = [pair[1] for pair in topic_dist]
topic_dist_table = pd.DataFrame()
topic_dist_table['Topic'] = topics
topic_dist_table['Probabilities'] = probabilities
topic_dist_table

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
fig.set_size_inches(11,4)
plt.bar(topic_dist_table['Topic'], topic_dist_table['Probabilities'], align='center', alpha=1, color='salmon')
plt.xlabel('topic')
plt.ylabel('probability')
plt.title('Per Topic Probability Distribution')
plt.show()

### Include the topic proportions $\theta_d$ for each house 


In [ ]:
num_topics = 10
theta = pd.DataFrame({"Theta0": np.zeros(ct_homes.shape[0])})
for t in np.arange(1,num_topics):
    theta["Theta"+str(t)] = np.zeros(ct_homes.shape[0])
    
for i in np.arange(ct_homes.shape[0]):
    for t in tm.get_document_topics(corpus[i]):
        theta.loc[i,"Theta"+str(t[0])] = t[1]

In [ ]:
ct_topics = ct_homes.join(theta)
ct_topics

#### Fit a linear model with the topic proportions included

We now fit a linear model with the topic proportions included. Note that 
the proportions satisfy $\theta_0+\theta_1+\cdots + \theta_9 = 1$. Therefore, we remove one of them, since it is redundant. If we don't do this the linear model will be harder to interpret!


In [ ]:
model = sm.ols("logPRICE ~ logAREA + logLOTSIZE + BED + BATH + BUILT + Theta0 + " +
               "Theta1 + Theta2 + Theta3 + Theta4 + Theta5 + Theta6 + Theta7 + Theta8", data=ct_topics).fit()
model.summary()

In [ ]:
plt.hist(model.resid, bins=50)
plt.show()

### Model without the topics included

In [ ]:
model_without_topics = sm.ols("logPRICE ~ logAREA + logLOTSIZE + BED + BATH + BUILT", data=ct_topics).fit()
model_without_topics.summary()

#### 2.2 Plot the residuals

On a single plot, show a histogram of the residuals of the model without the topics, 
and the residuals of the model with the topics. Give a legend that shows which is which.
Comment on the results. 


In [ ]:
# your code here

#### 2.3 Quantify the improvement: R-squared

How do the two models compare in terms of R-squared? What do these numbers mean?


[your markdown here]

#### 2.4 Quantify the improvement: MSE decrease

What is the percent decrease in the mean-squared-error of the model with the topics
compared to the model that ignores the descriptions?


In [ ]:
# [your code and markdown here]

#### 3.5 Quantify the improvement: LOOCV

What is the percent decrease in the leave-one-out-cross-validation (LOOCV) error?
Recall from class that the following formula can be used to calculate this:

<img src="https://raw.githubusercontent.com/YData123/sds265-fa21/main/assignments/assn6/loocv.png" width="410" align="center">

<br>

The following line of code computes this for one of the models:

`np.mean((model.resid/(1 - model.get_influence().hat_matrix_diag))**2)`

In [ ]:
# your code here 

#### 2.6 Repeat for three topics 

Now, repeat the above steps for a topic model that is trained using only three (3) topics. Specifically:

1. Train a model with three topics
1. Display the top words in each of the three topics
1. Augment the `ct_homes` data with the resulting topic proportions $\theta$
1. Fit a linear model *using only the first two of the three* proportions
1. Plot a histogram of the residuals of the three linear models together
1. Comment on the improvement over the baseline in terms of R-squared, MSE, and LOOCV compared with the previous two models.


In [ ]:
# your code and markdown here

#### 2.7 Interpretation

Now, interpret the model. Use the coefficients of the linear model to 
help interpret the meaning of the topics. Comment on what this says 
about the effectiveness of the topic model for predicting the sale price 
of the house. Does it make intuitive sense? Why or why not?


[your markdown here]

## Problem 3: Inside Out: Deep neural networks with tensorflow

In class, we discussed a "bare bones" implementation of a 2-layer neural network for classification, using rectified linear units as activation functions. 
This implementation only uses the `numpy` package. In practice, we don't need to implement each component of a neural network from scratch. There exists software libraries like Tensorflow and PyTorch which make the process of building and training neural networks much faster. At their core, such libraries implement automatic differentiation so that the use only needs to define the "forward pass" of their model, and, as long as they do so within the framework, the gradients can be computed automatically. In addition, these libraries also have many built-in definitions of common neural network components. [Tensorflow](https://www.tensorflow.org/) is an open-source package for building and training neural networks. We will use it in this problem to explore applying neural networks to some synthetic classification tasks.

Recall that a 2-layer neural network for classification takes the following form,
\begin{aligned}
    h_{1} &=\operatorname{ReLU}\left(W_{1} X+b_{1}\right) \\
    p &=\operatorname{Softmax}\left(W_{2} h_{1}+b_{2}\right).
\end{aligned}
where $X$ is the input. The number of layers is referred to as the "depth" of a neural network. We can construct neural networks with arbitrary depth via the following architecture,
\begin{aligned}
    h_{1} &=\operatorname{ReLU}\left(W_{1} X+b_{1}\right) \\
    h_{2} &=\operatorname{ReLU}\left(W_{2} h_1 + b_{2}\right) \\
    h_{3} &=\operatorname{ReLU}\left(W_{3} h_2 + b_{3}\right) \\
    &\vdots\\
    p &=\operatorname{Softmax}\left(W_{\ell} h_{\ell-1}+b_{\ell}\right).
\end{aligned}
This is an $\ell$-layer neural network. The $i$-th layer takes the output of the previous layer, $h_{i-1}$ as input, and produces $h_i$. The parameters of the $i$-th layer are the weights matrix $W_i$ and the bias vector $b_i$. The nonlinear activation function is $\operatorname{ReLU}$ at each hidden layer, and $\operatorname{Softmax}$ at the output layer.

In this problem, we will learn how to train deep neural networks with Tensorflow and apply them to synthetic classification tasks. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tqdm.keras import TqdmCallback

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
# plt.rcParams['axes.facecolor'] = 'lightgray'

First we generate and plot a spiral dataset that contains $K=3$ classes.

In [ ]:
def generate_spirals(N=100, K=3, noise=0.3):
    D = 2  # dimensionality
    X = np.zeros((N*K, D))
    y = np.zeros(N*K, dtype='uint8')
    for j in range(K):
        ix = range(N*j, N*(j+1))
        r = np.linspace(0.0, 1, N)  # radius
        t = np.linspace(j*4, (j+1)*4, N) + np.random.randn(N)*noise  
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = j
    return X, y

def plot_data(X, y):
    fig = plt.figure()
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.xlim(np.min(X[:,0])-.1, np.max(X[:,0])+.1)
    plt.ylim(np.min(X[:,1])-.1, np.max(X[:,1])+.1)


def plot_classifier(X, y, model):
    h = 0.015
    x_min, x_max = X[:, 0].min() - .2, X[:, 0].max() + .2
    y_min, y_max = X[:, 1].min() - .2, X[:, 1].max() + .2
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    # Z = np.dot(np.maximum(
    #     0, np.dot(np.c_[xx.ravel(), yy.ravel()], W1) + b1), W2) + b2
    inputs = np.array([xx.ravel(), yy.ravel()]).T
    Z = model(inputs)
    Z = np.argmax(Z, axis=1)
    Z = Z.reshape(xx.shape)
    fig = plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.5)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    
def plot_history(history, val=False, figsize=(8,3)):
    '''plot given attributes from training history'''
    plot_attrs = ('loss', 'accuracy')
    fig, axs = plt.subplots(ncols=len(plot_attrs), figsize=figsize)

    if not all(plot_attr in history.history for plot_attr in plot_attrs):
        raise ValueError('not all `plot_attrs` are in the history object')

    for plot_attr, ax in zip(plot_attrs, axs):
        ax.plot(history.history[plot_attr], label=plot_attr)
        if val:
            ax.plot(history.history[f'val_{plot_attr}'], label=f'val_{plot_attr}')
        ax.set_ylabel(plot_attr)
        ax.set_xlabel('epoch')
        ax.legend(loc='upper right')

    return fig

def train_neurnet(neurnet, X, y, X_val, y_val, n_epochs=1_000, batch_size=128):
    neurnet.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    history = neurnet.fit(
        X, y, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=batch_size, verbose=0,
        callbacks=[TqdmCallback(data_size = len(y), batch_size=batch_size, verbose=0)])
    plot_history(history, val=True);
    return history

In [ ]:
X_spiral, y_spiral = generate_spirals(N=300, K=3)
X_spiral_val, y_spiral_val = generate_spirals(N=50, K=3)
X_spiral_test, y_spiral_test = generate_spirals(N=300, K=3)
plot_data(X_spiral, y_spiral)

Now, let's train a 2-layer neural network to classify this dataset.

In [ ]:
# demo of neurnet
neurnet = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=3, activation='softmax')
    ])

neurnet.build(input_shape=(None, 2))
neurnet.summary()

In [ ]:
history = train_neurnet(neurnet, X_spiral, y_spiral, X_spiral_val, y_spiral_val, n_epochs=1_000)

In [ ]:
plot_classifier(X_spiral, y_spiral, neurnet)
plot_classifier(X_spiral_test, y_spiral_test, neurnet)

### Problem 3.1: Experiment with different architectures and hyperparameters settings

[Tensorflow playground](https://playground.tensorflow.org/) is a tool created by Tensorflow to demo how neural networks work, allowing users to tinker with different architectures on a few toy classification tasks (in fact, the same ones we are playing with in this problem). Under the hood, Tensorflow playground is running code like the one we are using in this notebook. Feel free to explore Tensorflow playground first before we continuing with this problem.

Using the above code, experiment with different architectures and hyperparameters of the optimization. In particular, try the following two variations.
1. Wider network. Increase the number of hidden units in the 2-layer network above. (hint: use the `units` argument in `layers.Dense`)
2. Deeper network. Increase the depth of the network by adding an additional hidden layer. (hint: add an additional `layers.Dense`)

For each variant, define your network using tensorflow, train it on the spiral data, visualize the decision boundaries, and include a markdown cell briefly describing what you observe. Feel free to add some additional variants as well. For example, you could train for longer, change the activation functions, etc.

**Wider network**

In [ ]:
# your code here

**Deeper network**

In [ ]:
# your code here

### Problem 3.2: Apply to three other toy datasets

Now, choose an architecture you like and apply your neural network to three more synthetic classification tasks generated by sklearn. For each of the following, define your neural network, train it on the dataset, and visualize the decision boundary. Similar to the above, feel free to experiment with different architectures to improve the performance of your neural network on each task.

#### "Circles"

In [ ]:
from sklearn import datasets

# generate circles
X_circles, y_circles = datasets.make_circles(300, noise=0.06, random_state=265)
X_circles_val, y_circles_val = datasets.make_circles(100, noise=0.06, random_state=565)
X_circles_test, y_circles_test = datasets.make_circles(300, noise=0.06, random_state=565)

plot_data(X_circles, y_circles)


The first toy dataset forms two concentered circles with different radius, and we add a little bit of noise so that two circles start mixing together. Train a neural network on this taks.

In [ ]:
# your code here

#### "Blobs"

In [ ]:
# generate blobs
centers = np.array([[-.5,.0],[0,.5], [.5,.1], [-.2, -.7], [.2, -.3]])
X_blobs, y_blobs = datasets.make_blobs(300, centers=centers, cluster_std=.15, center_box=(-1.0, 1.0), random_state=265)
X_blobs_val, y_blobs_val = datasets.make_blobs(100, centers=centers, cluster_std=.15, center_box=(-1.0, 1.0), random_state=565)
X_blobs_test, y_blobs_test = datasets.make_blobs(300, centers=centers, cluster_std=.15, center_box=(-1.0, 1.0), random_state=565)

plot_data(X_blobs, y_blobs)

The second dataset forms three clusters centered at different places. Repeat the procedure that you have walked through for the first dataset. Note that this time the number of classes is 5 not 3, so you'll have to adjust the number of units in the final unit accordingly.

In [ ]:
# Your code goes here

#### "Moons"

In [ ]:
# generate moons
X_moons, y_moons = datasets.make_moons(300, noise=0.2, random_state=265)
X_moons_val, y_moons_val = datasets.make_moons(100, noise=0.2, random_state=565)
X_moons_test, y_moons_test = datasets.make_moons(300, noise=0.2, random_state=565)

plot_data(X_moons, y_moons)

The third dataset is composed of two moon-shaped clusters. Repeat the procedures above on this dataset. Note that the number of classes is 2 now.

In [ ]:
# Your code goes here


<img src="https://upload.wikimedia.org/wikipedia/commons/5/53/Lake_Baikal_in_winter.jpg" width=300 style="padding: 10px; float: right;">


## Problem 4: Frozen: Navigating a random environment (25 points)

In class we introduce the Q-learning algorithm using the Taxi problem from the OpenAI `gym` package. In this problem we will explore another toy reinforcement learning problem, called  "Frozen Lake". In this problem you need to walk over a grid that represents a partially frozen lake, being careful not to fall through holes in the ice. (The author of this problem fell through the ice in a frozen lake when he was a kid&mdash;at night! It was quite an experience...) The environment is a simple 4x4 grid, and the goal is to walk from one corner to the other without falling through.

Unlike the Taxi problem, but more like Tic-Tac-Toe as discussed in class, there are no intermediate rewards. Rather the reward is 1 if you succeed, and 0 otherwise. `Frozen Lake` has two versions. In the first version, the ice is (unrealistically) not slippery. Here the state transitions are deterministic: If you move right, you go right. In the second version, the state transitions are probabilistic: If you try to move right, you may go left, down, or up. Naturally, the slippery version is more challenging. 

Your task in this problem will be to complete the implementation of the Q-learning algorithm for this problem, display the value function, and then evaluate the solution. You'll do this for both the deterministic and random (slippery) versions.

First load the necessary packages. You'll probably need to install `gym`, and can use `!pip install gym`. If you have difficulties let us know and we'll try to help.

In [ ]:
#!pip install gym

In [ ]:
import gym
import numpy as np
from IPython.display import clear_output
from time import sleep
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline


Here is the "ascii art" rendition of the starting state. You start in the upper left corner, marked `S`, and the goal is to get to the lower right corner, marked `G`. The ice has four holes, marked `H`; if you step here you fall through the ice and the episode is done. But as you learn, you do not know where the holes are. We'll first use the deterministic version, by specifying `is_slippery=False`.

This code was written and tested with `gym` version 0.26.2.

In [ ]:
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode='ansi')

Here are some simple helper functions, same as we used for the Taxi demo. Don't change the cell below, just run it.

In [ ]:
def render(env, stat, action, reward):
    return {'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward}


def print_frames(frames, delay=.1):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(delay)
        
def display_value_function(q_table):
    v = np.max(q_table, axis=1)
    plt.imshow(v.reshape(4,4))
    plt.axis('off')
    plt.colorbar()
    plt.show()
    print(np.round(v.reshape(4,4), 3))

def evaluate_Q_function(env, q_table, epsilon=.001, episodes=1000):
    total_steps, total_successes = 0, 0
    
    for _ in range(episodes):
        state, *_ = env.reset()
        steps, reward = 0, 0
        done = False
        while not done:
            if random.uniform(0, 1) < epsilon:
               action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values
    
            next_state, reward, done, *_ = env.step(action)
            state = next_state
            steps += 1

        total_successes += reward
        total_steps += steps
    
    print(f"Results after {episodes} episodes:")
    print(f"Average steps per episode: {total_steps / episodes}")
    print(f"Chance of success: {total_successes / episodes}")
    
def sample_episode(env, q_table, epsilon=.001):
    state, *_ = env.reset()
    steps, reward = 0, 0
    done = False
    frames = []
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        state, reward, done, *_ = env.step(action)
        frames.append(render(env, state, action, reward))
        steps += 1
    return frames

In the cell below, we simply walk around randomly, calling the `sample()` function to choose an action. At the end of the episode, the steps are displayed. If you're lucky, you'll make it to the goal; more likely is that you fall through the ice. Try it out a few times to make sure you understand how it works.

In [ ]:
env.reset()
steps = 0
reward = 0
frames = [] 

done = False
while not done:
    action = env.action_space.sample() # choose a random action
    state, reward, done, *_ = env.step(action)
    frames.append(render(env, state, action, reward))
    steps += 1
    

print_frames(frames, delay=.5)
print(f"\nSteps taken: {steps}, success: {reward}")


### Problem 4.1: Complete the Implementation of Q-learning

In the cell below we have started you off with a partial implementation of Q-learning. Your job is to finish the implementation. You'll then use your implementation to learn the Q-function for both the deterministic and random versions of the environment. 

The function declaration looks like this:

```python
def Q_learning(env, alpha=.1, gamma=.7, epsilon=.1, episodes=10000):
```

with the following arguments:

* `env` is the environment. We pass this in because we're going to have slippery and non-slippery versions.
* `alpha` is the step size
* `gamma` is the discount for future rewards
* `epsilon` is the probability of exploring
* `episodes` is the number of training episodes to use. 

Some hints:

* The Q-table is intialized to have all values 1/2. When you transition to a state and the value of `done` is `True`, there are two possibilities: (1) You fell through a hole in the ice or (2) you reached the goal (congratulations!)

* The value of the Q-function for these states (with all actions) should be zero; no future reward is possible.  You don't know where the holes in the ice are (even though you could "cheat" and read them off the rendering of the environment.

* If you reached the goal, which is state 15, the value of the reward when you transition to this state will be 1.

* All values of the Q-function should be less than or equal to 1---the value can be interpreted as the probability of reaching the goal starting from the state/action pair.

* You should only need 3-5 lines of code to complete the implementation! If you find yourself using more, you may want to rethink your approach.


In [ ]:
def Q_learning(env, alpha=.1, gamma=.7, epsilon=.1, episodes=10000):
    q_table = 0.5*np.ones([env.observation_space.n, env.action_space.n])
    for _ in tqdm(np.arange(episodes)):
        state, *_ = env.reset()
        done = False
        while not done:
            explore = False
            if random.uniform(0, 1) < epsilon:
                explore = True
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values
            next_state, reward, done, *_ = env.step(action) 
            ### Finish the implementation below. Only 3-5 lines of code needed.
            
    return q_table

### Problem 4.2

Now use the function `Q_learning` that you just wrote, and explore different settings 
of the parameters. The following cells carry out the following steps:

1. Create the environment
1. Run Q-learning. This is where you can experiment with different parameters
1. Display the value function `v(s) = np.max(q_table[s])`
1. Describe the value function, and how the numerical values make sense
1. Run `evaluate_Q_function` to get statistics on how well it works
1. Comment on the evaluation statistics
1. Print out a sample episode

The only line you need to change is the call to `Q_learning`, which is where you 
select the parameters. *Do not change the other cells*. You will be graded according 
to your implementation; these are checks to make sure it is working properly.

In [ ]:
# do not change
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode='ansi')

In [ ]:
# only change the following line, to set parameters
q_table = Q_learning(env, alpha=.1, gamma=.5, epsilon=.5, episodes=100)
display_value_function(q_table)

Describe the value function above. Do the numerical values in different states make sense? Why or why not? Explain.

*[your markdown here]*

In [ ]:
# just run this cell
evaluate_Q_function(env, q_table)

Do the evaluate statistics make sense? How do they compare to the random environment below? Explain.

*[your markdown here]*

In [ ]:
# just run this cell
frames = sample_episode(env, q_table)
print_frames(frames, delay=.1)

### Problem 4.3

Now use the function `Q_learning` on the random environment, 
where `is_slippery=True`. You may want to play around with this a bit 
to make sure you understand how it differs from the case where `is_slippery=False`.
The difference is that there is randomness in the state transitions for each 
action. If you try to go down, you may go right, for example.

As above, you run the following steps:

1. Create the environment
1. Run Q-learning. This is where you can experiment with different parameters
1. Display the value function `v(s) = np.max(q_table[s])`
1. Describe the value function, and how the numerical values make sense
1. Run `evaluate_Q_function` to get statistics on how well it works
1. Comment on the evaluation statistics
1. Print out a sample episode

The only line you need to change is the call to `Q_learning`, which is where you 
select the parameters. *Do not change the other cells*. You will be graded according 
to your implementation; these are checks to make sure it is working properly.

In [ ]:
# Do not change
random_env = gym.make("FrozenLake-v1", is_slippery=True, render_mode='ansi')

In [ ]:
# only change the following line, to set parameters
q_table = Q_learning(random_env, alpha=.1, gamma=.5, epsilon=.5, episodes=100)
display_value_function(q_table)

Describe the value function above. Do the numerical values in different states make sense? Why or why not? Explain.

*[your markdown here]*

In [ ]:
# just run this cell
evaluate_Q_function(random_env, q_table)

Do the output statistics make sense? How do they compare to the deterministic environment?Explain.

*[your markdown here]*

In [ ]:
# just run this cell
frames = sample_episode(random_env, q_table)
print_frames(frames, delay=.1)

*[Add any discussion of your implementation and findings that you wish to share. Nothing specific is required.]*

## Bonus question

What is the common theme of all of the problem names?